Bibliotecas utilizadas

In [ ]:
#Executar caso não tenha instalado:
!pip install xlsxwriter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 1.5 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import gzip as gz
import os
import xlsxwriter

In [ ]:
#Encontrar o arquivo:

path = '/content/drive/MyDrive/Mestrado/Estudos dirigidos/SimTelArray_2022_05'

#Todos os arquivos com excessão do gamma_diffuse tem um caminho: massa > images_tel > files .txt.gz/.dt.gz
#Aqui estamos interessados em trabalhar apenas com os arquivos .txt.gz que contém o sumário de cada uma das runs.

In [ ]:
#Descomprimir um arquivo txt:

def decompress_single(files_txt_list):
  file = files_txt_list[0] #pega apenas um arquivo

  with gz.open(file, 'rb') as f:
    data = f.read().decode("utf-8")

  data = data.split('\n@+')

  header = data[0].split('\n#@+')[1:-1] #Joga fora a primeira coluna e a última coluna que não serão interessantes

  rows_list = []

  for i in data[1:]:
    values = zip(header, i.split())
    rows_list.append(dict(values))

  df = pd.DataFrame(rows_list)

  return df

In [ ]:
def decompress_all(path):
  df_list_tel_1, df_list_tel_2, df_list_tel_3, df_list_tel_4 = [], [], [], [] #Listas de saída para com todas as runs e todos os telescópios.

  for file in path:
    rows_list1, rows_list2, rows_list3, rows_list4 = [], [], [], [] #Listas auxiliares para montar todos os dataframes.

    with gz.open(file, 'rb') as f: #Descompressão no arquivo .gz
      data = f.read().decode("utf-8") #Decodificação para o formato correto de arquivos

    data = data.split('\n@+') #Separação de aquivos de acordo com a estrutura dos dados recebidos
    header = data[0].split('\n#@+')[1:-1] #Joga fora a primeira coluna e a última coluna que não serão interessantes e define o cabeçalho dos arquivos.

    if 'tel_1' in file: #Verifica qual o telescópio que queremos salvar
      for i in data[1:]: #Percorre todas as linhas dentro da leitura ignorando a primeira linha que possui informações do documento.
        values = zip(header, i.split()) #Vínculo de cada um dos valores com seu respectivo Header
        rows_list1.append(dict(values)) #Adiciona à lista auxiliar os valores dentro da linha criando um mega diciinario no final do loop.
      df1 = pd.DataFrame(rows_list1) #Cria um dataframe que contém todas as informações sobre a run iterada
      df_list_tel_1.append(df1) #Adiciona a uma lista o dataframe da respectiva run na sua respectiva posição: tel_1_run_1 = df_list_tel_1[0]...tel_1_run_n = df_list_tel_1[n-1]

    if 'tel_2' in file: #Faz o mesmo que o primeiro IF
      for i in data[1:]:
        values = zip(header, i.split())
        rows_list2.append(dict(values))
      df2 = pd.DataFrame(rows_list2)
      df_list_tel_2.append(df2)

    if 'tel_3' in file: #Faz o mesmo que o primeiro IF
      for i in data[1:]:
        values = zip(header, i.split())
        rows_list3.append(dict(values))
      df3 = pd.DataFrame(rows_list3)
      df_list_tel_3.append(df3)

    if 'tel_4' in file: #Faz o mesmo que o primeiro IF
      for i in data[1:]:
        values = zip(header, i.split())
        rows_list4.append(dict(values))
      df4 = pd.DataFrame(rows_list4)
      df_list_tel_4.append(df4)

  return df_list_tel_1, df_list_tel_2, df_list_tel_3, df_list_tel_4 #Retorna cada lista com todas as runs de maneira independente.


In [ ]:
#Pega todos os tipos de massa dentro da pasta com exceção do arquivo texto e do gamma difuso.
mass_list = [i for i in os.listdir(path) if 'diffuse' not in i and 'Los' not in i]

In [ ]:
for i in mass_list:
  path_mass = os.path.join(path, i, 'images_tel') #Diretório da massa com todas as runs.
  files_txt = [os.path.join(path_mass, j) for j in os.listdir(path_mass) if 'txt' in j] #Salva todos os arquivos .txt.gz
  files_txt.sort() #Coloca tudo em ordem alfabética, ordenando todas as runs em ordem crescente.
  break #para na primeira iteração.

In [ ]:
decompress_single(files_txt)

,(1): event,(2): telescope,(3): energy,(4): core distance to telescope,(5): image size (amplitude) [p.e.],(6): number of pixels in image,(7): width [deg.],(8): length [deg.],(9): distance [deg.],(10): miss [deg.],(11): alpha [deg.],(12): orientation [deg.],(13): direction [deg.],(14): image c.o.g. x [deg.],(15): image c.o.g. y [deg.],(16): Xmax [g/cm^2],(17): Hmax [m],(18): Npe (true number of photo-electrons)
0,800,1,0.1087,130.65,625.40,16,0.04718,0.14921,0.70891,-1.00000,5.62953,244.376,238.747,-0.307,-0.639,296.316,9865.892,812
1,1200,1,0.0616,99.54,245.10,12,0.08901,0.18506,0.74829,-1.00000,10.96088,229.106,240.067,-0.490,-0.566,301.579,9743.963,380
2,1700,1,0.0904,68.77,445.92,16,0.06645,0.11386,0.55937,-1.00000,0.51109,223.119,43.630,-0.408,-0.382,320.000,9330.016,764
3,2300,1,4.5563,233.85,5381.81,67,0.08451,0.37075,1.75286,-1.00000,1.21828,113.385,294.604,-0.696,1.609,439.919,7015.053,6283
4,3600,1,0.3297,386.63,31.70,2,-1.00000,-1.00000,1.75286,-1.00000,1.21828,113.385,294.604,-0.696,1.609,232.857,11491.405,179
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2236,999200,1,0.0358,66.11,82.27,5,0.05595,0.08490,0.11946,-1.00000,55.42740,256.858,132.285,-0.027,-0.116,190.000,12805.976,128
2237,999500,1,0.1038,62.89,435.53,16,0.06059,0.17611,0.39649,-1.00000,11.28212,74.922,63.640,0.103,0.383,186.667,12918.216,600
2238,999700,1,1.0562,350.95,237.49,14,0.05773,0.16775,1.97074,-1.00000,6.46152,207.288,20.827,-1.751,-0.904,284.351,10149.576,339
2239,999800,1,0.1367,250.87,151.31,11,0.04312,0.31467,1.71896,-1.00000,4.28978,117.330,121.620,-0.789,1.527,296.000,9873.262,312


In [ ]:
df_list_tel_1, df_list_tel_2, df_list_tel_3, df_list_tel_4 = decompress_all(files_txt)

In [ ]:
len(df_list_tel_1),len(df_list_tel_2),len(df_list_tel_3),len(df_list_tel_4)
#100 runs em cada uma das listas

In [ ]:
electron = [df_list_tel_1, df_list_tel_2, df_list_tel_3, df_list_tel_4]

In [ ]:
def save_single_csv(dir_out, single_telescope):
  # Criar um objeto ExcelWriter para salvar várias abas
  with pd.ExcelWriter(dir_out, engine='xlsxwriter') as writer:
      for i, df in enumerate(single_telescope):
          # Nomeando as abas como 'run_1', 'run_2',...
          df.to_excel(writer, sheet_name=f'run_{i+1}', index=False)

In [ ]:
save_single_csv(path_out, electron[0])

In [ ]:
def save_mass_summary(dir_out, mass, mass_type):
  for name, telescope in enumerate(mass):
    dir = dir_out
    dir = os.path.join(dir_out, f'{mass_type}_tel_{name+1}.xlsx')

    with pd.ExcelWriter(dir, engine='xlsxwriter') as writer:
          for i, df in enumerate(telescope):
              # Nomeando as abas como 'run_1', 'run_2',...
              df.to_excel(writer, sheet_name=f'run_{i+1}', index=False)

In [ ]:
dir_summary = '/content/drive/MyDrive/Mestrado/Estudos dirigidos/summary'
dir_summary_mass = os.path.join(dir_summary, 'electron')

In [ ]:
dir_summary_mass

'/content/drive/MyDrive/Mestrado/Estudos dirigidos/summary/electron'

In [ ]:
save_mass_summary(dir_summary_mass, electron, 'electron')

/content/drive/MyDrive/Mestrado/Estudos dirigidos/summary/electron/tel_1.xlsx
/content/drive/MyDrive/Mestrado/Estudos dirigidos/summary/electron/tel_2.xlsx
/content/drive/MyDrive/Mestrado/Estudos dirigidos/summary/electron/tel_3.xlsx
/content/drive/MyDrive/Mestrado/Estudos dirigidos/summary/electron/tel_4.xlsx


In [ ]:
all_mass_path = {} #Esse dicionário deve existir antes de rodar essa função:

def get_mass_paths(mass, path, all_mass_path):
  path_mass = os.path.join(path, mass, 'images_tel') #Diretório da massa com todas as runs.

  files_txt = [os.path.join(path_mass, j) for j in os.listdir(path_mass) if 'txt' in j] #Salva todos os arquivos .txt.gz

  files_txt.sort() #Coloca tudo em ordem alfabética, ordenando todas as runs em ordem crescente.

  all_mass_path[mass] = files_txt

In [ ]:
all_mass_path = {} #Garantia de que o dicionário esteja vazio

for mass in mass_list:
  get_mass_paths(mass, path, all_mass_path)

In [ ]:
dir_summary = '/content/drive/MyDrive/Mestrado/Estudos dirigidos/summary'

In [ ]:
mass_summary = {}

for mass in mass_list:
  df_list_tel_1, df_list_tel_2, df_list_tel_3, df_list_tel_4 = decompress_all(all_mass_path[mass])

  mass_summary[mass] = [df_list_tel_1, df_list_tel_2, df_list_tel_3, df_list_tel_4]

  dir_summary_mass = os.path.join(dir_summary, mass)

  save_mass_summary(dir_summary_mass, mass_summary[mass], mass)

Exception ignored in: <function ZipFile.__del__ at 0x79d800880820>
Traceback (most recent call last):
  File "/usr/lib/python3.10/zipfile.py", line 1821, in __del__
  File "/usr/lib/python3.10/zipfile.py", line 1838, in close
ValueError: seek of closed file


In [ ]:
mass_summary['gamma'][0][99] #[Massa][Telescopio][run_#]

,(1): event,(2): telescope,(3): energy,(4): core distance to telescope,(5): image size (amplitude) [p.e.],(6): number of pixels in image,(7): width [deg.],(8): length [deg.],(9): distance [deg.],(10): miss [deg.],(11): alpha [deg.],(12): orientation [deg.],(13): direction [deg.],(14): image c.o.g. x [deg.],(15): image c.o.g. y [deg.],(16): Xmax [g/cm^2],(17): Hmax [m],(18): Npe (true number of photo-electrons)
0,100,1,2.8502,59.20,28256.89,65,0.09106,0.31080,0.75045,-1.00000,2.30791,350.837,353.145,0.741,-0.120,340.769,8885.213,40266
1,300,1,3.0629,400.79,73.49,6,0.05202,0.09468,2.18757,-1.00000,10.15547,347.745,177.901,2.138,-0.464,391.067,7890.007,139
2,700,1,4.0950,171.86,12991.43,60,0.07310,0.34168,1.24795,-1.00000,0.00278,46.553,46.550,0.858,0.906,316.486,9407.493,16962
3,800,1,2.6162,423.13,168.89,11,0.06027,0.15362,2.02338,-1.00000,3.80544,195.145,18.950,-1.953,-0.529,282.134,10203.183,271
4,900,1,3.3945,485.32,71.29,5,0.04252,0.11597,2.17258,-1.00000,11.47382,118.586,287.113,-1.040,1.908,282.836,10186.176,120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2964,399500,1,1.0244,400.45,11.33,1,-1.00000,-1.00000,1.45814,-1.00000,0.21137,111.554,111.765,-0.536,1.356,292.330,9959.370,84
2965,399700,1,2.9609,350.46,888.51,35,0.08036,0.25134,2.03062,-1.00000,1.31682,6.233,187.550,2.019,0.220,314.000,9462.733,1183
2966,399800,1,4.7034,415.93,267.68,14,0.06227,0.16268,2.11397,-1.00000,10.04587,0.772,190.818,2.114,0.028,332.055,9069.179,367
2967,399900,1,4.0264,347.66,1163.68,35,0.08283,0.24029,2.02640,-1.00000,1.00576,128.787,309.793,-1.269,1.580,298.667,9811.221,1471
